In [1]:
import pandas as pd
import json
import glob
from src import iids_util
from sklearn.metrics import (cohen_kappa_score)
from itertools import combinations

### Evaluating IIDS

In [2]:
def get_evaluator(scenario, layer ,dev, target_class):
    key = f"output/{layer.title()}-IIDS_S{scenario}_{dev}_{target_class}.json"
    with open(key, 'r') as f:
        metric = json.load(f)
    del key
    key = f"output/{layer.title()}-IIDS_S{scenario}_{dev}_{target_class}.csv"
    output = pd.read_csv(key, low_memory=False)
    return (output, metric)

### Decision's Reliability
1. Agreement score between Edge-IIDS and Fog-IIDS
2. Agreement score inter-model in an IIDS

In [56]:
# Scenario-1
scenario = 3
e_out, e_met = get_evaluator(scenario=scenario, layer='edge', dev='fridge', target_class='normal')
f_out, f_met = get_evaluator(scenario=scenario, layer='fog', dev='fridge', target_class='normal')

In [57]:
# Agreement Score between Edge-IIDS and Fog-IIDS
agg_score = cohen_kappa_score(y1=e_out['y_predict'].tolist(),
                              y2=f_out['y_predict'].tolist())
print("Aggrement Score Edge-IIDS vs Fog-IIDS: ", f'{agg_score:.3f}')

Aggrement Score Edge-IIDS vs Fog-IIDS:  0.611


In [58]:
# Inter-Model in Edge-IIDS
learner = e_out.drop(columns=['y_predict', 'y_true']).columns.tolist()
scores = e_out[learner].to_numpy()
preds = [iids_util.proba_prediction_rules(nscore=i) for i in scores]
yname = [f'y{num}' for num in range(1, len(learner)+1)]
e_out[yname] = preds
del scores, preds

In [59]:
for pair in combinations(yname, 2):
    score = cohen_kappa_score(y1=e_out[pair[0]].tolist(),
                              y2=e_out[pair[1]].tolist())
    msg = f'Agreement score {pair[0]} vs {pair[1]}: {score:.2f}'
    print(msg)

Agreement score y1 vs y2: -0.43
Agreement score y1 vs y3: -0.55
Agreement score y1 vs y4: -0.15
Agreement score y1 vs y5: -0.23
Agreement score y1 vs y6: -0.26
Agreement score y2 vs y3: 0.39
Agreement score y2 vs y4: 0.07
Agreement score y2 vs y5: 0.09
Agreement score y2 vs y6: 0.05
Agreement score y3 vs y4: 0.05
Agreement score y3 vs y5: 0.10
Agreement score y3 vs y6: 0.12
Agreement score y4 vs y5: 0.69
Agreement score y4 vs y6: 0.73
Agreement score y5 vs y6: 0.77


In [54]:
# Inter-Model in Fog-IIDS
learner = f_out.drop(columns=['y_predict', 'y_true']).columns.tolist()
scores = f_out[learner].to_numpy()
preds = [iids_util.proba_prediction_rules(nscore=i) for i in scores]
yname = [f'y{num}' for num in range(1, len(learner)+1)]
f_out[yname] = preds
del scores, preds

In [55]:
for pair in combinations(yname, 2):
    score = cohen_kappa_score(y1=f_out[pair[0]].tolist(),
                              y2=f_out[pair[1]].tolist())
    msg = f'Agreement score {pair[0]} vs {pair[1]}: {score:.2f}'
    print(msg)

Agreement score y1 vs y2: 0.89
Agreement score y1 vs y3: 0.92
Agreement score y2 vs y3: 0.91
